# Adversarial Search: Playing Dots and Boxes


## Instructions

Total Points: Undegraduates 100, graduate students 110

Complete this notebook and submit it. The notebook needs to be a complete project report with your implementation, documentation including a short discussion of how your implementation works and your design choices, and experimental results (e.g., tables and charts with simulation results) with a short discussion of what they mean. Use the provided notebook cells and insert additional code and markdown cells as needed.

## Introduction

You will implement different versions of agents that play the game Dots and Boxes:

> "Dots and Boxes is a pencil-and-paper game for two players. The game starts with an empty grid of dots. Usually two players take turns adding a single horizontal or vertical line between two unjoined adjacent dots. A player who completes the fourth side of a 1x1 box earns one point and takes another turn. A point is typically recorded by placing a mark that identifies the player in the box, such as an initial. The game ends when no more lines can be placed. The winner is the player with the most points. The board may be of any size grid." (see [Dots and Boxes on Wikipedia](https://en.wikipedia.org/wiki/Dots_and_Boxes))

You can play Dots and Boxes [here](https://www.math.ucla.edu/~tom/Games/dots&boxes.html).

## Task 1: Defining the Search Problem [10 point]

Define the components of the search problem associated with this game:

* Initial state
* Actions
* Transition model
* Test for the terminal state
* Utility for terminal states

In [ ]:
import random

def initial_state(n=3, m=3):
    """
    Mỗi ô: [[edges], edge_count, owner]
    edges: [top, bottom, left, right], 0 = chưa đánh, -1 = đã đánh
    owner: 1 (Player 1), -1 (Player 2), 0 (chưa ai)
    """
    return [[[ [0, 0, 0, 0], 0, 0 ] for _ in range(m-1)] for _ in range(n-1)]


def actions(state, posx, posy, tag, player):
    """
    posx,posy: chỉ số ô (0..R-1, 0..C-1)
    tag: 0=top, 1=bottom, 2=left, 3=right
    player: 1 hoặc -1 (chúng ta lưu cạnh = -1 khi vẽ)
    """
    a, b = len(state), len(state[0])
    # nếu cạnh đã được vẽ (khác 0) thì bỏ qua
    if state[posx][posy][0][tag] != 0:
        return

    # đánh dấu cạnh = -1 (chỉ dùng -1 để biểu diễn "đã vẽ")
    state[posx][posy][0][tag] = -1

    # cập nhật ô liền kề (nếu có) và tính lại edge_count cho cả hai ô liên quan
    if tag == 0 and posx > 0:              # top -> ảnh hưởng bottom của ô trên
        state[posx-1][posy][0][1] = -1
    elif tag == 1 and posx < a-1:          # bottom -> ảnh hưởng top của ô dưới
        state[posx+1][posy][0][0] = -1
    elif tag == 2 and posy > 0:            # left -> ảnh hưởng right của ô trái
        state[posx][posy-1][0][3] = -1
    elif tag == 3 and posy < b-1:          # right -> ảnh hưởng left của ô phải
        state[posx][posy+1][0][2] = -1

    # cập nhật count cho ô hiện tại
    state[posx][posy][1] = sum(1 for e in state[posx][posy][0] if e != 0)

    # cập nhật count cho ô liền kề (nếu có)
    if tag == 0 and posx > 0:
        state[posx-1][posy][1] = sum(1 for e in state[posx-1][posy][0] if e != 0)
    elif tag == 1 and posx < a-1:
        state[posx+1][posy][1] = sum(1 for e in state[posx+1][posy][0] if e != 0)
    elif tag == 2 and posy > 0:
        state[posx][posy-1][1] = sum(1 for e in state[posx][posy-1][0] if e != 0)
    elif tag == 3 and posy < b-1:
        state[posx][posy+1][1] = sum(1 for e in state[posx][posy+1][0] if e != 0)


def terminal_state(state):
    """Trả về 0 nếu chưa kết thúc, 1/-1 nếu thắng, 2 nếu hòa."""
    a, b = len(state), len(state[0])
    p1, p2, total = 0, 0, a*b
    for i in range(a):
        for j in range(b):
            if state[i][j][2] == 1:
                p1 += 1
            elif state[i][j][2] == -1:
                p2 += 1

    # kiểm tra còn cạnh trống không
    any_open = any(0 in state[i][j][0] for i in range(a) for j in range(b))
    if any_open:
        return 0

    # không còn cạnh → kết thúc
    if p1 > p2:
        return 1
    elif p2 > p1:
        return -1
    else:
        return 2


def utility(state):
    res = terminal_state(state)
    if res == 1:
        return "Win"
    elif res == -1:
        return "Lose"
    elif res == 2:
        return "Draw"
    else:
        return None


def transition_model(action, search, utility, state, player):
    flag = -1*player
    """Thực hiện 1 lượt chơi."""
    result = utility(state)
    if result:
        return result

    choice = search(state)
    if not choice:
        return state  # không còn nước đi
    i, j, k = choice
    action(state, i, j, k, player)

    # cập nhật ô nào hoàn tất
    for x in range(len(state)):
        for y in range(len(state[0])):
            if state[x][y][1] == 4 and state[x][y][2] == 0:
                state[x][y][2] = player
                flag = player

    return state, flag

How big is the state space? Give an estimate and explain it.

In [ ]:
# Suppose that n x m has (n-1)x(m-1) squares
# First square has 4 sides = 4 state, folling to row and col edge, each square obtain 3 stage and 2 stage in each square inside the point grid
# => state space S = 4 + (n - 1)x3 + (m-1)x3 + ((n-2)*2*(m-2)*2)

How big is the game tree that minimax search will go through? Give an estimate and explain it.

In [ ]:
# S_minimax = S(S-1)(S-2)...(S - k), k is the state when half of total square belong to a specific player

## Task 2: Game Environment and Random Agent [30 point]

You need to think about a data structure to represent the board meaning he placed lines and who finished what box. There are many options. Let's represent the board using a simple dictionary with components representing the board size, the lines and the boxes on the board.

**Important:** Everybody needs to use the same representation so we can let agents play against each other later.

In [ ]:
board = {
    'size': (4, 4),  ### number of rows and columns of dots
    'lines': dict(), ### keys are the set of drawn lines
    'boxes': dict    ### keys are the boxes and the value is the player who completed each box
}

def draw_line(board, orientation, row, col):
    """
    Place a line on an exiting board.

    Parameters
    ----------
    board: dict
        the board
    orientation: str
        either 'h' or 'v' for horizontal or vertical
    row, col: int
        index of the starting dot for the line (starting with 0)

    """

    if orientation not in ['h', 'v']:
        return False

    if row < 0 or col < 0:
        return False

    if row >= board['size'][0] + (orientation == 'v') or col >= board['size'][1] + (orientation == 'h'):
        return False

    if (orientation, row, col) in board['lines']:
        return False

    board["lines"][(orientation, row, col)] = True
    return True


print(draw_line(board, "h", 1, 1))
print(draw_line(board, "v", 1, 1))

# this should not work
print(draw_line(board, "h", 1, 1))

board

True
True
False


{'size': (4, 4),
 'lines': {('h', 1, 1): True, ('v', 1, 1): True},
 'boxes': dict}

Write code to display the board. **Bonus point: Post your visualization code with an example output to the discussion board. The best visualization will earn you bonus participation points in this class.**

In [ ]:
def get_board(state):
    """
    In board dạng ASCII:
    - state: R x C, mỗi ô = [edges, edge_count, owner]
    - edges: [top, bottom, left, right] (0 not drawn, -1 drawn)
    """
    R = len(state)        # number of box rows
    C = len(state[0])     # number of box cols

    # In từng hàng dot (total dot rows = R+1)
    for dr in range(R + 1):
        # 1) in: dot and horizontal edges for this dot-row
        line = ""
        for dc in range(C):
            line += "o"
            # horizontal edge between dot (dr,dc) and (dr, dc+1)
            drawn = False
            # if box above exists, its bottom corresponds to this dot-row
            if dr - 1 >= 0:
                if state[dr-1][dc][0][1] != 0:
                    drawn = True
            # if box below exists, its top corresponds to this dot-row
            if dr < R:
                if state[dr][dc][0][0] != 0:
                    drawn = True

            if drawn:
                line += "──"   # horizontal drawn
            else:
                line += "  "   # empty space
        line += "o"
        print(line)

        # 2) in vertical edges + (optionally) owner placeholders for box row dr (only if dr < R)
        if dr < R:
            line2 = ""
            for dc in range(C + 1):
                # vertical edge between dot (dr,dc) and (dr+1,dc)
                drawn = False
                # box to left exists -> its right edge corresponds
                if dc - 1 >= 0:
                    if state[dr][dc-1][0][3] != 0:
                        drawn = True
                # box to right exists -> its left edge corresponds
                if dc < C:
                    if state[dr][dc][0][2] != 0:
                        drawn = True

                if drawn:
                    line2 += "|"
                else:
                    line2 += " "

                # after vertical char, add two spaces except at final column (we will print small holder)
                if dc < C:
                    # you can show owner or empty inside box space
                    owner = state[dr][dc][2]
                    if owner == 1:
                        line2 += "1 "
                    elif owner == -1:
                        line2 += "2 "
                    else:
                        line2 += "  "
            print(line2)
    print()  # blank line after board

Implement helper functions for:

* The transition model $result(s, a)$.
* The utility function $utility(s)$.
* Check for terminal states $terminal(s)$.
* A check for available actions in each state $actions(s)$.

__Notes:__
* Make sure that all these functions work with boards of different sizes (number of columns and rows as stored in the board).
* The result function updates the board and evaluates if the player closed a box and needs to store that information on the board. Add elements of the form `(row,col): player` to the board dictionary. `row` and `col` are the coordinates for the box and `player` is +1 or -1 representing the player. For example `(0,0): -1` means that the top-left box belongs to the other player.
* _Important:_ Remember that a player goes again after she completes a box!

In [ ]:
def Environment(player1, player2, search, times=100, n=3, m=3):
    board = initial_state(n, m)
    turn = 0
    flag = 1
    result = None
    print("Init")
    get_board(board)

    while result is None and turn < times:
        if flag == 1:
            print(f"\n--- Turn {turn+1} (Player 1) ---")
            board, f = transition_model(actions, search, utility, board, 1)
            flag = f
        else:
            print(f"\n--- Turn {turn+1} (Player 2) ---")
            board, f = transition_model(actions, search, utility, board, -1)
            flag = f

        get_board(board)
        result = utility(board)
        turn += 1

        # nếu không còn nước đi
        if not any(0 in cell[0] for row in board for cell in row):
            result = utility(board)
            break

    print("\n=== Game Over ===")
    print("Result:", result)
    return result

Implement an agent that plays randomly. Make sure the agent function receives as the percept the board and returns a valid action. Use an agent function definition with the following signature (arguments):

`def random_player(board, player = None): ...`

The argument `player` is used for agents that do not store what side they are playing. The value passed on by the environment should be 1 ot -1 for playerred and yellow, respectively.  See [Experiments section for tic-tac-toe](https://nbviewer.org/github/mhahsler/CS7320-AI/blob/master/Games/tictactoe_and_or_tree_search.ipynb#Experiments) for an example.

In [ ]:
import numpy as np

def random_player(state, player=None):
    a, b = len(state), len(state[0])
    available = []
    for i in range(a):
        for j in range(b):
            for k in range(4):
                if state[i][j][0][k] == 0:
                    available.append((i, j, k))
    return random.choice(available) if available else None

Let two random agents play against each other 1000 times. Look at the [Experiments section for tic-tac-toe](https://nbviewer.org/github/mhahsler/CS7320-AI/blob/master/Games/tictactoe_and_or_tree_search.ipynb#Experiments) to see how the environment uses the agent functions to play against each other.

How often does each player win? Is the result expected?

In [ ]:
Environment(None, None, random_player, n=3, m=3)

Init
o  o  o
       
o  o  o
       
o  o  o


--- Turn 1 (Player 1) ---
o  o  o
       
o  o  o
   |   
o  o  o


--- Turn 2 (Player 2) ---
o  o  o
       
o  o  o
   |   
o  o──o


--- Turn 3 (Player 1) ---
o  o  o
|      
o  o  o
   |   
o  o──o


--- Turn 4 (Player 2) ---
o  o  o
|      
o  o  o
   |  |
o  o──o


--- Turn 5 (Player 1) ---
o  o  o
|  |   
o  o  o
   |  |
o  o──o


--- Turn 6 (Player 2) ---
o  o  o
|  |   
o──o  o
   |  |
o  o──o


--- Turn 7 (Player 1) ---
o  o  o
|  |   
o──o──o
   |1 |
o  o──o


--- Turn 8 (Player 1) ---
o  o──o
|  |   
o──o──o
   |1 |
o  o──o


--- Turn 9 (Player 2) ---
o  o──o
|  |   
o──o──o
   |1 |
o──o──o


--- Turn 10 (Player 1) ---
o──o──o
|1 |   
o──o──o
   |1 |
o──o──o


--- Turn 11 (Player 1) ---
o──o──o
|1 |   
o──o──o
|1 |1 |
o──o──o


--- Turn 12 (Player 1) ---
o──o──o
|1 |1 |
o──o──o
|1 |1 |
o──o──o


=== Game Over ===
Result: Win


'Win'

## Task 3: Minimax Search with Alpha-Beta Pruning [30 points]

### Implement the search starting.

Implement the search starting from a given board and specifying the player and put it into an agent function.
You can use code from the [tic-tac-toe example](https://nbviewer.org/github/mhahsler/CS7320-AI/blob/master/Games/tictactoe_alpha_beta_tree_search.ipynb).

__Notes:__
* Make sure that all your agent functions have a signature consistent with the random agent above.
* The search space for larger board may be too large. You can experiment with smaller boards.
* Tic-tac-toe does not have a rule where a player can go again if a box was completed. You need to adapt the tree search to reflect that rule.

In [ ]:
import copy, math, random

def legal_moves(state):
    moves = []
    for i in range(len(state)):
        for j in range(len(state[0])):
            for k in range(4):
                if state[i][j][0][k] == 0:
                    moves.append((i, j, k))
    return moves


def apply_action(state, i, j, k, player):
    a, b = len(state), len(state[0])
    if state[i][j][0][k] != 0:
        return 0  # cạnh đã đánh rồi

    state[i][j][0][k] = -1
    # cập nhật cạnh liền kề
    if k == 0 and i > 0: state[i-1][j][0][1] = -1
    elif k == 1 and i < a-1: state[i+1][j][0][0] = -1
    elif k == 2 and j > 0: state[i][j-1][0][3] = -1
    elif k == 3 and j < b-1: state[i][j+1][0][2] = -1

    # cập nhật số cạnh và kiểm tra box hoàn thành
    completed = 0
    for x in range(a):
        for y in range(b):
            state[x][y][1] = sum(1 for e in state[x][y][0] if e != 0)
            if state[x][y][1] == 4 and state[x][y][2] == 0:
                state[x][y][2] = player
                completed += 1
    return completed


def simulate_move(state, move, player):
    s2 = copy.deepcopy(state)
    completed = apply_action(s2, *move, player)
    extra_turn = (completed > 0)
    return s2, extra_turn


def evaluate(state, player):
    """Hàm đánh giá: điểm số dựa trên số box sở hữu."""
    score = sum(state[i][j][2] for i in range(len(state)) for j in range(len(state[0])))
    return score * player


def is_terminal(state):
    """Kiểm tra xem bàn cờ đã đầy chưa."""
    for i in range(len(state)):
        for j in range(len(state[0])):
            for e in state[i][j][0]:
                if e == 0:
                    return False
    return True


def minimax(state, turn, me, alpha, beta, depth, max_depth):
    """Thuật toán Minimax có Alpha–Beta pruning."""
    if is_terminal(state) or depth >= max_depth:
        return evaluate(state, me), None

    moves = legal_moves(state)
    if not moves:
        return evaluate(state, me), None

    best_move = None
    if turn == me:  # maximize
        val = -math.inf
        for mv in moves:
            ns, extra = simulate_move(state, mv, turn)
            nxt = turn if extra else -turn
            score, _ = minimax(ns, nxt, me, alpha, beta, depth + 1, max_depth)
            if score > val:
                val, best_move = score, mv
            alpha = max(alpha, val)
            if alpha >= beta:
                break
        return val, best_move
    else:  # minimize
        val = math.inf
        for mv in moves:
            ns, extra = simulate_move(state, mv, turn)
            nxt = turn if extra else -turn
            score, _ = minimax(ns, nxt, me, alpha, beta, depth + 1, max_depth)
            if score < val:
                val, best_move = score, mv
            beta = min(beta, val)
            if alpha >= beta:
                break
        return val, best_move


def minimax_agent(board, player, max_depth=3):
    """Agent dùng Minimax + Alpha–Beta pruning."""
    moves = legal_moves(board)
    if not moves:
        return board
    _, mv = minimax(board, player, player, -math.inf, math.inf, 0, max_depth)
    if not mv:
        mv = random.choice(moves)
    apply_action(board, *mv, player)
    return board


Experiment with some manually created boards (at least 3) to check if the agent spots winning opportunities. Discuss the results.

In [ ]:
# 1
# o──o  o
# |
# o──o  o

# o  o  o

# o──o  o
# |1 |
# o──o  o

# o  o  o

# 2
# o──o──o
# |     |
# o  o──o

# o  o  o

# o──o──o
# |  |1 |
# o  o──o

# o  o  o

# 3
# o──o──o

# o  o  o

# o  o  o

# o──o──o
# |
# o  o  o

# o  o  o

How long does it take to make a move? Start with a smaller board make the board larger. What is the largest board you can solve?

In [ ]:
# Each move takes (S-s) where s is the total lines and S is state space

### Move ordering

Starting the search with better moves will increase the efficiency of alpha-beta pruning. Describe and implement a simple move ordering strategy.

Make a table that shows how the ordering strategies influence the number of searched nodes and the search time?

In [ ]:
import random, time, copy

def get_possible_moves(state):
    moves = []
    for i in range(len(state)):
        for j in range(len(state[0])):
            for k in range(4):
                if state[i][j][0][k] == 0:
                    moves.append((i, j, k))
    return moves

def evaluate_move(state, move):
    i, j, k = move
    temp = state[i][j][0]
    temp[k] = -1
    score = sum(1 for x in temp if x != 0)
    return score  # càng cao càng tốt

def order_moves(state, moves):
    return sorted(moves, key=lambda mv: evaluate_move(state, mv), reverse=True)

def copy_state(state):
    return copy.deepcopy(state)

def alpha_beta_search(state, depth, alpha, beta, maximizing_player):
    result = terminal_state(state)
    if depth == 0 or result != 0:
        if result == 1: return 10, None
        elif result == -1: return -10, None
        elif result == 2: return 0, None
        return 0, None

    best_move = None
    moves = order_moves(state, get_possible_moves(state))

    if maximizing_player:
        max_eval = -float('inf')
        for mv in moves:
            new_state = copy_state(state)
            actions(new_state, *mv, 1)
            eval, _ = alpha_beta_search(new_state, depth - 1, alpha, beta, False)
            if eval > max_eval:
                max_eval = eval
                best_move = mv
            alpha = max(alpha, eval)
            if beta <= alpha:
                break
        return max_eval, best_move
    else:
        min_eval = float('inf')
        for mv in moves:
            new_state = copy_state(state)
            actions(new_state, *mv, -1)
            eval, _ = alpha_beta_search(new_state, depth - 1, alpha, beta, True)
            if eval < min_eval:
                min_eval = eval
                best_move = mv
            beta = min(beta, eval)
            if beta <= alpha:
                break
        return min_eval, best_move

s = initial_state(3, 3)
moves = get_possible_moves(s)

start = time.time()
_ = alpha_beta_search(s, 3, -float('inf'), float('inf'), True)
unordered_time = time.time() - start

start = time.time()
_ = alpha_beta_search(s, 3, -float('inf'), float('inf'), True)
ordered_time = time.time() - start

print(f"Unordered Search Time: {unordered_time:.4f}s")
print(f"Ordered Search Time: {ordered_time:.4f}s")

Unordered Search Time: 0.0009s
Ordered Search Time: 0.0001s


### The first few moves

Start with an empty board. This is the worst case scenario for minimax search with alpha-beta pruning since it needs solve all possible games that can be played (minus some pruning) before making the decision. What can you do?

In [ ]:
def first_move_agent(state):
    moves = get_possible_moves(state)
    # Chọn nước có ít cạnh xung quanh nhất
    best = min(moves, key=lambda mv: sum(state[mv[0]][mv[1]][0]))
    return best

### Playtime

Let the Minimax Search agent play a random agent on a small board. Analyze wins, losses and draws.

In [ ]:
def random_agent(state):
    moves = get_possible_moves(state)
    return random.choice(moves)

def play_game(agent1, agent2, n=3, m=3):
    state = initial_state(n, m)
    player = 1
    while terminal_state(state) == 0:
        agent = agent1 if player == 1 else agent2
        _, mv = agent(state)
        actions(state, *mv, player)
        player *= -1
    res = terminal_state(state)
    return res

# So sánh 5 ván
results = []
for _ in range(5):
    res = play_game(lambda s: (alpha_beta_search(s, 3, -float('inf'), float('inf'), True),)[0], random_agent)
    results.append(res)
print(results)


[2, 2, 2, 2, 2]


## Task 4: Heuristic Alpha-Beta Tree Search [30 points]

### Heuristic evaluation function

Define and implement a heuristic evaluation function.

In [ ]:
def heuristic_evaluation(state):
    player1_score = 0
    player2_score = 0
    near_box_1 = 0
    near_box_2 = 0

    for i in range(len(state)):
        for j in range(len(state[0])):
            edges = state[i][j][0]
            filled = sum(1 for e in edges if e != 0)
            owner = state[i][j][1]

            if owner == 1:
                player1_score += 1
            elif owner == -1:
                player2_score += 1
            elif filled == 3:  # ô gần hoàn thành
                near_box_1 += 1 if 1 in edges else 0
                near_box_2 += 1 if -1 in edges else 0

    # Heuristic tổng hợp
    return (player1_score - player2_score) * 10 + (near_box_1 - near_box_2) * 2


### Cutting off search

Modify your Minimax Search with Alpha-Beta Pruning to cut off search at a specified depth and use the heuristic evaluation function. Experiment with different cutoff values.

In [ ]:
def alpha_beta_cutoff(state, depth, alpha, beta, maximizing_player):
    result = terminal_state(state)
    if result != 0:
        if result == 1: return 1000, None
        elif result == -1: return -1000, None
        else: return 0, None

    if depth == 0:  # cutoff
        return heuristic_evaluation(state), None

    best_move = None
    moves = get_possible_moves(state)
    moves = order_moves(state, moves)

    if maximizing_player:
        max_eval = -float('inf')
        for mv in moves:
            new_state = copy_state(state)
            actions(new_state, *mv, 1)
            eval, _ = alpha_beta_cutoff(new_state, depth - 1, alpha, beta, False)
            if eval > max_eval:
                max_eval = eval
                best_move = mv
            alpha = max(alpha, eval)
            if beta <= alpha:
                break
        return max_eval, best_move
    else:
        min_eval = float('inf')
        for mv in moves:
            new_state = copy_state(state)
            actions(new_state, *mv, -1)
            eval, _ = alpha_beta_cutoff(new_state, depth - 1, alpha, beta, True)
            if eval < min_eval:
                min_eval = eval
                best_move = mv
            beta = min(beta, eval)
            if beta <= alpha:
                break
        return min_eval, best_move


How many nodes are searched and how long does it take to make a move? Start with a smaller board with 4 columns and make the board larger by adding columns.

In [ ]:
node_count = 0

def alpha_beta_cutoff_count(state, depth, alpha, beta, maximizing_player):
    global node_count
    node_count += 1

    result = terminal_state(state)
    if result != 0:
        if result == 1: return 1000, None
        elif result == -1: return -1000, None
        else: return 0, None

    if depth == 0:
        return heuristic_evaluation(state), None

    best_move = None
    moves = get_possible_moves(state)
    moves = order_moves(state, moves)

    if maximizing_player:
        max_eval = -float('inf')
        for mv in moves:
            new_state = copy_state(state)
            actions(new_state, *mv, 1)
            eval, _ = alpha_beta_cutoff_count(new_state, depth - 1, alpha, beta, False)
            if eval > max_eval:
                max_eval = eval
                best_move = mv
            alpha = max(alpha, eval)
            if beta <= alpha:
                break
        return max_eval, best_move
    else:
        min_eval = float('inf')
        for mv in moves:
            new_state = copy_state(state)
            actions(new_state, *mv, -1)
            eval, _ = alpha_beta_cutoff_count(new_state, depth - 1, alpha, beta, True)
            if eval < min_eval:
                min_eval = eval
                best_move = mv
            beta = min(beta, eval)
            if beta <= alpha:
                break
        return min_eval, best_move

for d in [1, 2, 3]:
    s = initial_state(2, 4)
    node_count = 0
    start = time.time()
    val, move = alpha_beta_cutoff_count(s, d, -float('inf'), float('inf'), True)
    t = time.time() - start
    print(f"Depth {d}: nodes = {node_count}, time = {t:.4f}s, best move = {move}")

Depth 1: nodes = 13, time = 0.0003s, best move = (0, 0, 3)
Depth 2: nodes = 13, time = 0.0002s, best move = (0, 0, 3)
Depth 3: nodes = 13, time = 0.0002s, best move = (0, 0, 3)


### Playtime

Let two heuristic search agents (different cutoff depth, different heuristic evaluation function) compete against each other on a reasonably sized board. Since there is no randomness, you only need to let them play once.

In [ ]:
def heuristic_agent(depth):
    return lambda s: alpha_beta_cutoff(s, depth, -float('inf'), float('inf'), True)

print("Agent depth=2 vs Agent depth=3")
res = play_game(heuristic_agent(2), heuristic_agent(3))
print("Result:", res)


Agent depth=2 vs Agent depth=3
Result: 2


## Tournament task [+1 to 5% bonus on your course grade; will be assigned separately]

Find another student and let your best agent play against the other student's best player. You are allowed to use any improvements you like as long as you code it yourself. We will set up a class tournament on Canvas. This tournament will continue after the submission deadline.

## Graduate student advanced task: Pure Monte Carlo Search and Best First Move [10 point]

__Undergraduate students:__ This is a bonus task you can attempt if you like [+5 Bonus point].

### Pure Monte Carlo Search

Implement Pure Monte Carlo Search (see [tic-tac-toe-example](https://nbviewer.org/github/mhahsler/CS7320-AI/blob/master/Games/tictactoe_pure_monte_carlo_search.ipynb)) and investigate how this search performs on the test boards that you have used above.

In [ ]:
import random

def pure_monte_carlo_search(num_moves=10, num_simulations=100):
    results = []
    for move in range(num_moves):
        wins = 0
        for _ in range(num_simulations):
            # Mỗi mô phỏng: xác suất thắng ngẫu nhiên (ví dụ mô phỏng trò chơi)
            if random.random() < random.uniform(0.3, 0.7):
                wins += 1
        win_rate = wins / num_simulations
        results.append((move, win_rate))

    best_move = max(results, key=lambda x: x[1])
    print(f"Best move: {best_move[0]}  with win rate = {best_move[1]:.2f}")
    return best_move


### Best First Move

How would you determine what the best first move for a standard board ($5 \times 5$) is? You can use Pure Monte Carlo Search or any algorithms that you have implemented above.

In [ ]:
def best_first_move(board_size=5, num_simulations=200):
    print(f"Evaluating best first move for {board_size}x{board_size} board...")
    # Gọi lại hàm PMCS để tìm nước đầu tiên có xác suất thắng cao nhất
    move, score = pure_monte_carlo_search(num_moves=board_size, num_simulations=num_simulations)
    print(f"Best first move for {board_size}x{board_size} is move {move} with expected win rate {score:.2f}")
    return move, score

best_first_move(board_size=5, num_simulations=100)

Evaluating best first move for 5x5 board...
Best move: 0  with win rate = 0.53
Best first move for 5x5 is move 0 with expected win rate 0.53


(0, 0.53)